In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import matplotlib.pyplot as plt
from FCMAE_model import FCMAE, ConvNeXtBlock1D


/home/nwertheim/miniconda3/bin/python
/home/nwertheim/miniconda3/bin/python


In [2]:
train_file = os.path.join(univariate_data_path, 'merged_univariate_train.npy')
train_data = np.load(train_file, allow_pickle=True)
test_file = os.path.join(univariate_data_path, 'merged_univariate_test.npy')
test_data = np.load(test_file, allow_pickle=True)
print(train_data.shape)
print(test_data.shape)

print(train_data[0]['signal'].shape)
print(train_data[0])



(532,)
(134,)
(72200, 1)
{'record_name': 'ice002_p_2of3', 'signal': array([[-12.15116958],
       [-24.48972151],
       [-18.22349939],
       ...,
       [  3.40956282],
       [  1.22642183],
       [  4.90008321]]), 'metadata': {'fs': 20, 'sig_len': 746000, 'n_sig': 16, 'base_date': None, 'base_time': None, 'units': ['mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV'], 'comments': ['Info:', 'ID:ice002', 'Record type:pregnancy', 'Record number:2/3', 'Age(years):38', 'BMI before pregnancy:20.7', 'BMI at recording:25.9', 'Gravidity:4', 'Parity:1', 'Previous caesarean:No', 'Placental position:Posterior', 'Gestational age at recording(w/d):39/1', 'Gestational age at delivery:40/4', 'Mode of delivery:Vaginal', 'Synthetic oxytocin use in labour:No', 'Epidural during labour:No', 'Comments for recording:', 'Reference on right hip - apparent reverse ECG.', 'Comments for delivery:']}}


In [3]:
import numpy as np
import pandas as pd

# Function to create time windows from the signal data
def create_time_windows(data, window_length=12000, step_size=12000):
    """
    Create sliding windows of a specified length and step size from a list of signals.

    Args:
        data: list of dicts, each containing a 'signal' array of shape (seq_len, 1)
        window_length: number of time steps in each window
        step_size: number of time steps to move between windows (for overlap)

    Returns:
        np.ndarray: shape (num_windows, window_length), univariate windows
    """
    windows = []

    for entry in data:
        signal = entry['signal'].flatten()  # shape: (seq_len,)
        signal_length = len(signal)

        if signal_length < window_length:
            continue

        for start_idx in range(0, signal_length - window_length + 1, step_size):
            end_idx = start_idx + window_length
            window = signal[start_idx:end_idx]
            windows.append(window)

    return np.array(windows)  # shape: (num_windows, window_length)


X_train = create_time_windows(train_data)
X_test = create_time_windows(test_data)
print(X_train.shape)
print(X_test.shape)

(1230, 12000)
(320, 12000)


In [4]:
# import numpy as np
# from sklearn.model_selection import train_test_split

# def split_data(windows, test_size=0.1, val_size=0.1, random_seed=42):
#     """
#     Splits the dataset into training, validation, and test sets with specified proportions.
    
#     Parameters:
#     - windows (numpy.ndarray): The dataset to split, shaped as (num_samples, window_size).
#     - test_size (float): Proportion of data to be used for the test set.
#     - val_size (float): Proportion of data to be used for the validation set.
#     - random_seed (int): Seed for reproducibility.

#     Returns:
#     - X_train, X_val, X_test: The splits of the dataset.
#     """
#     # First, split into train and temp (test + validation)
#     X_train, X_temp = train_test_split(windows, test_size=(test_size + val_size), random_state=random_seed)
    
#     # Then, split the temp into validation and test
#     val_size_adjusted = val_size / (val_size + test_size)  # Adjust to split remaining temp
#     X_val, X_test = train_test_split(X_temp, test_size=val_size_adjusted, random_state=random_seed)
    
#     return X_train, X_val, X_test

# # Example usage with your windows data
# X_train, X_val, X_test = split_data(windows, test_size=0.1, val_size=0.1)

# # Check the shapes of the splits
# print(f"Training set shape: {X_train.shape}")
# print(f"Validation set shape: {X_val.shape}")
# print(f"Test set shape: {X_test.shape}")


In [5]:
def mask_data(x, mask_ratio=0.5, patch_size=8):
    """
    Apply patch-based masking to a batch of univariate time series.

    Args:
        x: np.ndarray of shape (num_windows, window_length)
        mask_ratio: float, fraction of patches to mask
        patch_size: int, number of time steps in each patch

    Returns:
        np.ndarray: masked version of x with same shape
    """
    x_masked = np.copy(x)
    num_windows, window_length = x.shape
    num_patches = window_length // patch_size

    for i in range(num_windows):
        mask = np.random.rand(num_patches) < mask_ratio
        for j in range(num_patches):
            if mask[j]:
                start = j * patch_size
                end = (j + 1) * patch_size
                x_masked[i, start:end] = 0  # or np.nan if you prefer

    return x_masked

# Apply masking to train, validation, and test sets
mask_ratio = 0.75  # Adjust this to your desired masking ratio
masked_X_train = mask_data(X_train, mask_ratio)
masked_X_test = mask_data(X_test, mask_ratio)

# Check the shape of the masked datasets
print(f"Masked Training set shape: {masked_X_train.shape}")
print(f"Masked Test set shape: {masked_X_test.shape}")

Masked Training set shape: (1230, 12000)
Masked Test set shape: (320, 12000)


In [6]:
# import matplotlib.pyplot as plt

# def plot_masked_examples(original, masked, num_examples=5):
#     plt.figure(figsize=(15, num_examples * 2.5))
    
#     for i in range(num_examples):
#         # Original signal
#         plt.subplot(num_examples, 2, 2*i + 1)
#         plt.plot(original[i], color='blue')
#         plt.title(f"Original Signal {i+1}")
#         plt.xlabel("Time step")
#         plt.ylabel("Amplitude")

#         # Masked signal
#         plt.subplot(num_examples, 2, 2*i + 2)
#         plt.plot(masked[i], color='orange')
#         plt.title(f"Masked Signal {i+1}")
#         plt.xlabel("Time step")
#         plt.ylabel("Amplitude")

#     plt.tight_layout()
#     plt.show()
# plot_masked_examples(windows, masked_windows, num_examples=5)


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.model_selection import KFold
import optuna
import numpy as np

# Your FCMAE definition here (same as you provided above)
# Your ConvNeXtBlock1D if needed

# Assume masked_X_train is already a NumPy array
masked_X_train_tensor = torch.tensor(masked_X_train, dtype=torch.float32).unsqueeze(1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def objective(trial):
    # Hyperparameter suggestions
    num_blocks = trial.suggest_int("num_blocks", 2, 5)
    kernel_size = trial.suggest_categorical("kernel_size", [3, 5, 7, 9])
    base_dim = trial.suggest_categorical("base_dim", [32, 64, 128])
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    fold_losses = []
    print('Trial: ', trial)
    for train_index, val_index in kf.split(masked_X_train_tensor):
        train_dataset = TensorDataset(masked_X_train_tensor[train_index])
        val_dataset = TensorDataset(masked_X_train_tensor[val_index])

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        model = FCMAE(in_channels=1, base_dim=base_dim, num_blocks=num_blocks, kernel_size=kernel_size).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.MSELoss()

        # Train for a few epochs (keep low for speed)
        for epoch in range(5):
            model.train()
            for batch in train_loader:
                inputs = batch[0].to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, inputs)
                loss.backward()
                optimizer.step()

        # Evaluate on validation fold
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch[0].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, inputs)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        fold_losses.append(avg_val_loss)

    return np.mean(fold_losses)

# Start Optuna tuning
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=25)

# Best result
print("Best trial:")
print(study.best_trial)
path = os.path.join(models_path, 'fcmae_cv_optuna.pkl')

# Optionally save the study for later
import joblib
joblib.dump(study, path)


/home/nwertheim/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-04-23 15:45:57,221] A new study created in memory with name: no-name-e9d72477-265b-4215-965d-095d6b359e40


Trial:  <optuna.trial._trial.Trial object at 0x14df03380ec0>


[I 2025-04-23 15:59:03,723] Trial 0 finished with value: 0.0011366075050318614 and parameters: {'num_blocks': 3, 'kernel_size': 3, 'base_dim': 32, 'learning_rate': 0.00016091546580773128, 'batch_size': 32}. Best is trial 0 with value: 0.0011366075050318614.


Trial:  <optuna.trial._trial.Trial object at 0x14de532a5880>
